In [1]:
import os
import pandas as pd
import datetime
import urllib.request
import json

In [2]:
df_station = pd.read_csv('data/df_station.csv')  
df_station

,id,location,name
0,02t,แขวงหิรัญรูจี เขตธนบุรี กทม.,มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา
1,05t,แขวงบางนา เขตบางนา กทม.,กรมอุตุนิยมวิทยา
2,10t,แขวงคลองจั่น เขตบางกะปิ กทม.,เคหะชุมชนคลองจั่น
3,11t,แขวงดินแดง เขตดินแดง กทม.,สนามกีฬาเคหะชุมชนห้วยขวาง
4,12t,แขวงช่องนนทรี เขตยานนาวา กทม.,โรงเรียนนนทรีวิทยา
...,...,...,...
82,63t,ต.สะเตง อ.เมือง จ.ยะลา,สวนสาธารณะสนามโรงพิธีช้างเผือก ยะลา
83,78t,ต.เบตง อ.เบตง จ.ยะลา,ศูนย์พัฒนาเด็กเล็กเทศบาลเมืองเบตง
84,80t,ต.พิมาน อ.เมือง. จ.สตูล,ศาลากลางจังหวัดสตูล
85,89t,ต.คลัง อ.เมือง จ.นครศรีธรรมราช,ศูนย์ฟื้นฟูสุขภาพผู้สูงอายุ


In [3]:
station_id = df_station.id.tolist()
type(station_id)

list

In [4]:
len(station_id)

87

In [5]:
station_id[:5]

['02t', '05t', '10t', '11t', '12t']

In [6]:
stations = station_id
measurements=['PM25', 'PM10', 'O3', 'CO', 'NO2', 'SO2', 'WS', 'WD', 'TEMP', 'RH', 'BP', 'RAIN']
sdate='2024-01-01'
edate='2023-01-31'

In [7]:
URL = 'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}' +  \
     '&param={measurements}&type=hr&sdate={sdate}&edate={edate}'+ \
     '&stime=00&etime=24'
URL

'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}&param={measurements}&type=hr&sdate={sdate}&edate={edate}&stime=00&etime=24'

In [8]:
station_groups = [stations[i:i+5] for i in range(0, len(stations), 5)]
station_groups

[['02t', '05t', '10t', '11t', '12t'],
 ['59t', '61t', '03t', '50t', '52t'],
 ['53t', '54t', '08t', '16t', '17t'],
 ['18t', '19t', '13t', '22t', '14t'],
 ['27t', '20t', '81t', '21t', '24t'],
 ['25t', '26t', '79t', '84t', '85t'],
 ['99t', '100t', '46t', '47t', '72t'],
 ['82t', '83t', '88t', '90t', '91t'],
 ['101t', '102t', '28t', '29t', '30t'],
 ['31t', '74t', '32t', '33t', '34t'],
 ['60t', '71t', '77t', '87t', '100t'],
 ['35t', '36t', '37t', '38t', '39t'],
 ['40t', '41t', '57t', '73t', '58t'],
 ['67t', '75t', '68t', '69t', '70t'],
 ['76t', '86t', '92t', '94t', '95t'],
 ['96t', '97t', '98t', '42t', '43t'],
 ['44t', '62t', '63t', '78t', '80t'],
 ['89t', '93t']]

In [9]:
records = []
for group in station_groups:
    # Construct the URL for this group of stations
    url = URL.format(station_ids=','.join(group), measurements=','.join(measurements), sdate=sdate, edate=edate)
    
    # Fetch data from the API
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())
        
    # Process the data and add it to the records
    for s in data['stations']:
        for r in s['data']:
            r['DATETIMEDATA'] = records.append(dict(stationID=s['stationID'], **r))

In [10]:
records[:2]

[{'stationID': '02t',
  'DATETIMEDATA': '2023-12-01 00:00:00',
  'PM25': 33,
  'PM10': None,
  'O3': None,
  'CO': None,
  'NO2': None,
  'SO2': None,
  'WS': 0.7,
  'WD': 49,
  'TEMP': 30,
  'RH': 53,
  'BP': 754,
  'RAIN': 0},
 {'stationID': '02t',
  'DATETIMEDATA': '2023-12-01 01:00:00',
  'PM25': 41,
  'PM10': None,
  'O3': None,
  'CO': None,
  'NO2': None,
  'SO2': None,
  'WS': 0.8,
  'WD': 51,
  'TEMP': 29.5,
  'RH': 55,
  'BP': 753,
  'RAIN': 0}]

In [11]:
type(records)

list

In [12]:
df = pd.DataFrame(records)
df

,stationID,DATETIMEDATA,PM25,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,02t,2023-12-01 00:00:00,33.0,NaN,NaN,NaN,NaN,NaN,0.7,49,30.0,53,754,0.0
1,02t,2023-12-01 01:00:00,41.0,NaN,NaN,NaN,NaN,NaN,0.8,51,29.5,55,753,0.0
2,02t,2023-12-01 02:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,5,29.1,57,753,0.0
3,02t,2023-12-01 03:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,30,28.5,60,753,0.0
4,02t,2023-12-01 04:00:00,36.0,NaN,NaN,NaN,NaN,NaN,0.7,19,28.1,62,753,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64810,93t,2023-12-31 20:00:00,17.0,31.0,22.0,0.18,2.0,NaN,2.7,60,29.0,81,754,0.0
64811,93t,2023-12-31 21:00:00,16.0,29.0,22.0,0.18,2.0,1.0,2.7,41,28.5,83,755,0.0
64812,93t,2023-12-31 22:00:00,14.0,25.0,23.0,0.16,1.0,1.0,2.8,43,28.5,83,755,0.0
64813,93t,2023-12-31 23:00:00,12.0,22.0,24.0,0.18,1.0,1.0,2.8,22,28.2,85,755,0.0


In [13]:
df.columns = ['id', 'datetime', 'pm25', 'pm10', 'o3', 'co', 'no2', 'so2', 'ws', 'wd', 'temp', 'rh', 'bp', 'rain']
df

,id,datetime,pm25,pm10,o3,co,no2,so2,ws,wd,temp,rh,bp,rain
0,02t,2023-12-01 00:00:00,33.0,NaN,NaN,NaN,NaN,NaN,0.7,49,30.0,53,754,0.0
1,02t,2023-12-01 01:00:00,41.0,NaN,NaN,NaN,NaN,NaN,0.8,51,29.5,55,753,0.0
2,02t,2023-12-01 02:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,5,29.1,57,753,0.0
3,02t,2023-12-01 03:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,30,28.5,60,753,0.0
4,02t,2023-12-01 04:00:00,36.0,NaN,NaN,NaN,NaN,NaN,0.7,19,28.1,62,753,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64810,93t,2023-12-31 20:00:00,17.0,31.0,22.0,0.18,2.0,NaN,2.7,60,29.0,81,754,0.0
64811,93t,2023-12-31 21:00:00,16.0,29.0,22.0,0.18,2.0,1.0,2.7,41,28.5,83,755,0.0
64812,93t,2023-12-31 22:00:00,14.0,25.0,23.0,0.16,1.0,1.0,2.8,43,28.5,83,755,0.0
64813,93t,2023-12-31 23:00:00,12.0,22.0,24.0,0.18,1.0,1.0,2.8,22,28.2,85,755,0.0


In [14]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time
df

,id,datetime,pm25,pm10,o3,co,no2,so2,ws,wd,temp,rh,bp,rain,date,time
0,02t,2023-12-01 00:00:00,33.0,NaN,NaN,NaN,NaN,NaN,0.7,49,30.0,53,754,0.0,2023-12-01,00:00:00
1,02t,2023-12-01 01:00:00,41.0,NaN,NaN,NaN,NaN,NaN,0.8,51,29.5,55,753,0.0,2023-12-01,01:00:00
2,02t,2023-12-01 02:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,5,29.1,57,753,0.0,2023-12-01,02:00:00
3,02t,2023-12-01 03:00:00,34.0,NaN,NaN,NaN,NaN,NaN,0.4,30,28.5,60,753,0.0,2023-12-01,03:00:00
4,02t,2023-12-01 04:00:00,36.0,NaN,NaN,NaN,NaN,NaN,0.7,19,28.1,62,753,0.0,2023-12-01,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64810,93t,2023-12-31 20:00:00,17.0,31.0,22.0,0.18,2.0,NaN,2.7,60,29.0,81,754,0.0,2023-12-31,20:00:00
64811,93t,2023-12-31 21:00:00,16.0,29.0,22.0,0.18,2.0,1.0,2.7,41,28.5,83,755,0.0,2023-12-31,21:00:00
64812,93t,2023-12-31 22:00:00,14.0,25.0,23.0,0.16,1.0,1.0,2.8,43,28.5,83,755,0.0,2023-12-31,22:00:00
64813,93t,2023-12-31 23:00:00,12.0,22.0,24.0,0.18,1.0,1.0,2.8,22,28.2,85,755,0.0,2023-12-31,23:00:00


In [15]:
df = df.drop(columns=['datetime'])
df

,id,pm25,pm10,o3,co,no2,so2,ws,wd,temp,rh,bp,rain,date,time
0,02t,33.0,NaN,NaN,NaN,NaN,NaN,0.7,49,30.0,53,754,0.0,2023-12-01,00:00:00
1,02t,41.0,NaN,NaN,NaN,NaN,NaN,0.8,51,29.5,55,753,0.0,2023-12-01,01:00:00
2,02t,34.0,NaN,NaN,NaN,NaN,NaN,0.4,5,29.1,57,753,0.0,2023-12-01,02:00:00
3,02t,34.0,NaN,NaN,NaN,NaN,NaN,0.4,30,28.5,60,753,0.0,2023-12-01,03:00:00
4,02t,36.0,NaN,NaN,NaN,NaN,NaN,0.7,19,28.1,62,753,0.0,2023-12-01,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64810,93t,17.0,31.0,22.0,0.18,2.0,NaN,2.7,60,29.0,81,754,0.0,2023-12-31,20:00:00
64811,93t,16.0,29.0,22.0,0.18,2.0,1.0,2.7,41,28.5,83,755,0.0,2023-12-31,21:00:00
64812,93t,14.0,25.0,23.0,0.16,1.0,1.0,2.8,43,28.5,83,755,0.0,2023-12-31,22:00:00
64813,93t,12.0,22.0,24.0,0.18,1.0,1.0,2.8,22,28.2,85,755,0.0,2023-12-31,23:00:00


In [16]:
df.to_csv('data/air_pollution_2024.csv', index=False)

if os.path.exists('data/air_pollution_2024.csv'):
    print("export success")
else:
    print("export failed")

export success
